In [1]:
import cv2
import torch
import numpy as np

c:\Users\helmi\anaconda3\envs\yolov5a\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
customyolov5s = torch.hub.load('','custom', path='weightHedect/nextgen_hedec_s2.pt', source='local')

YOLOv5  v6.1-224-g3931687 Python-3.8.13 torch-1.11.0+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
#colorcode
clrYellow = (38,247,255)
clrGreen = (0,255,0)
clrRed = (255,0,0)

In [4]:
ori_img = cv2.imread('test_image/constructon-worker-wellfare.jpg')

cv2.imshow('This img',ori_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
print(device)

Setup complete. Using torch 1.11.0+cpu (CPU)
cpu


In [6]:
model = customyolov5s.to(device)

In [7]:
img = ori_img
results = model(img)

In [9]:
print(results.pandas().xyxy[0])

         xmin        ymin        xmax        ymax  confidence  class    name
0   94.184029  132.789978  178.732758  228.621429    0.939074      0    head
1   94.137985  132.941681  180.425995  179.959579    0.888721      1  helmet
2  439.412872   37.721931  534.322083  131.212357    0.788271      1  helmet
3  440.039520   37.660103  548.706299  143.153671    0.714847      0    head


In [8]:
# print(results.pandas().xyxy[0])

presults = results.pandas().xyxy[0]


results_cpu = results.xyxyn[0].cpu().numpy()

i = 0

# print(presults.iloc[:,:-1])

print(presults)
print(presults.loc[3])


# print(results_cpu)

         xmin        ymin        xmax        ymax  confidence  class    name
0   94.184029  132.789978  178.732758  228.621429    0.939074      0    head
1   94.137985  132.941681  180.425995  179.959579    0.888721      1  helmet
2  439.412872   37.721931  534.322083  131.212357    0.788271      1  helmet
3  440.039520   37.660103  548.706299  143.153671    0.714847      0    head
xmin           440.03952
ymin           37.660103
xmax          548.706299
ymax          143.153671
confidence      0.714847
class                  0
name                head
Name: 3, dtype: object


In [9]:
#add center to dataframe of result


newcol_xycenter = []

for i in range(len(presults)):
    newcol_xycenter.append((
        int((presults["xmin"][i]+presults["xmax"][i])/2),int((presults["ymin"][i]+presults["ymax"][i])/2)
    ))

print(newcol_xycenter)

presults2 = presults

presults2["xycenter"] = newcol_xycenter

print(presults2)

[(136, 180), (137, 156), (486, 84), (494, 90)]
         xmin        ymin        xmax        ymax  confidence  class    name  \
0   94.184029  132.789978  178.732758  228.621429    0.939074      0    head   
1   94.137985  132.941681  180.425995  179.959579    0.888721      1  helmet   
2  439.412872   37.721931  534.322083  131.212357    0.788271      1  helmet   
3  440.039520   37.660103  548.706299  143.153671    0.714847      0    head   

     xycenter  
0  (136, 180)  
1  (137, 156)  
2   (486, 84)  
3   (494, 90)  


In [10]:
def bb_intersection_over_union(presults, nameA,nameB):
	boxA = [presults['xmin'][nameA],presults['ymin'][nameA],presults['xmax'][nameA],presults['ymax'][nameA]]
	boxB = [presults['xmin'][nameB],presults['ymin'][nameB],presults['xmax'][nameB],presults['ymax'][nameB]]

	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def check_helmetWear(presults):
    head_list = []
    hardhat_list = []
    pair_result = []
    for i in range(len(presults)):
        if presults['class'][i] == 0:
            hardhat_list.append(i)
        elif presults['class'][i] == 1:
            head_list.append(i)


    pair_result = []
    for i in range(len(head_list)):
        # hardhat_iou_scores = []
        besthardhat = []
        for j in range(len(hardhat_list)):
            iou_result = bb_intersection_over_union(presults,head_list[i],hardhat_list[j])
            
            if len(besthardhat) == 0:
                besthardhat = [hardhat_list[j],iou_result]
                continue

            if besthardhat[1] < iou_result:
                besthardhat = [hardhat_list[j],iou_result]

        pair_result.append([head_list[i],besthardhat[0],besthardhat[1]])

    return pair_result


def print_result(m_presults,m_pairResult,plain_img,wearTresh):
    #colorCoding
    no_helmet_detected = 0
    printedimg = plain_img
    
    
    # print helmet
    for i in range(len(m_presults)):
        if m_presults["class"][i] == 1:
            printedimg = cv2.rectangle(printedimg, (int(m_presults['xmin'][i]),int(m_presults['ymin'][i])),(int(m_presults['xmax'][i]),int(m_presults['ymax'][i])),clrYellow,1)
            
            
    for i in range(len(m_pairResult)):
        if m_pairResult[i][2] < wearTresh:
            no_helmet_detected += 1
            printedimg = cv2.rectangle(printedimg, (int(m_presults['xmin'][m_pairResult[i][0]]),int(m_presults['ymin'][m_pairResult[i][0]])),(int(m_presults['xmax'][m_pairResult[i][0]]),int(m_presults['ymax'][m_pairResult[i][0]])),clrRed,2)
            printedimg = cv2.putText(printedimg,"head",(int(m_presults['xmin'][m_pairResult[i][0]]),int(m_presults['ymin'][m_pairResult[i][0]])),cv2.FONT_HERSHEY_SIMPLEX,0.9,clrRed,2)
        else :
            printedimg = cv2.rectangle(printedimg, (int(m_presults['xmin'][m_pairResult[i][0]]),int(m_presults['ymin'][m_pairResult[i][0]])),(int(m_presults['xmax'][m_pairResult[i][0]]),int(m_presults['ymax'][m_pairResult[i][0]])),clrGreen,2)
            printedimg = cv2.putText(printedimg,"head",(int(m_presults['xmin'][m_pairResult[i][0]]),int(m_presults['ymin'][m_pairResult[i][0]])),cv2.FONT_HERSHEY_SIMPLEX,0.9,clrGreen,2)

            printedimg = cv2.rectangle(printedimg, (int(m_presults['xmin'][m_pairResult[i][1]]),int(m_presults['ymin'][m_pairResult[i][1]])),(int(m_presults['xmax'][m_pairResult[i][1]]),int(m_presults['ymax'][m_pairResult[i][1]])),clrGreen,2)
            printedimg = cv2.putText(printedimg,"helmet",(int(m_presults['xmin'][m_pairResult[i][1]]),int(m_presults['ymin'][m_pairResult[i][1]])),cv2.FONT_HERSHEY_SIMPLEX,0.9,clrGreen,2)
            
            printedimg = cv2.line(printedimg,m_presults['xycenter'][m_pairResult[i][0]],m_presults['xycenter'][m_pairResult[i][1]],clrGreen,2)
    
    return printedimg
    
            

In [12]:
check_helmetWear(presults)

0.4781444114541731
0.0
0.0
0.9223915611187468


In [11]:
pair_result = check_helmetWear(presults2)

print(pair_result)

img = ori_img

printedimg = print_result(presults2,pair_result,img,0.1)



[[1, 0, 0.4909419179595239], [2, 3, 0.7670195416284188]]


In [12]:
cv2.imshow('This img',printedimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# cv2.rectangle(img, (115,228),(281,415),(0, 255, 0),2)

img = ori_img

def printTheBoxYEAAAH(inimg,predresultinpandas):
    printed_img = inimg
    for i in range(len(predresultinpandas)):
        printed_img = cv2.rectangle(printed_img, (int(predresultinpandas['xmin'][i]),int(predresultinpandas['ymin'][i])),(int(predresultinpandas['xmax'][i]),int(predresultinpandas['ymax'][i])),(0, 255, 0),2)
        printed_img = cv2.putText(printed_img,predresultinpandas['name'][i],(int(predresultinpandas['xmin'][i]),int(predresultinpandas['ymin'][i])),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0, 255, 0),2)

    return printed_img
    


printedimg = printTheBoxYEAAAH(img,presults)

cv2.imshow('This img',printedimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
cv2.imshow('This img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()